# DataParallel
引入 PyTorch 模块和定义参数。  

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
from tqdm import tqdm

## 并行处理示例
### 常用参数

In [2]:
input_size = 5
output_size = 2
batch_size = 30
data_size = 100
# 设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### 随机生成数据集
随机生成一个数据集集，并且实现访问元素和返回长度两个方法。创建data_loader。

In [3]:
class RandomDataset(Dataset): # 继承Dataset
    def __init__(self,size,length):
        self.len = length
        self.data = torch.randn(length,size)
    
    def __getitem__(self,index): # 访问元素
        return self.data[index]
    
    def __len__(self): # 返回长度
        return self.len
# 创建数据集加载器，维度为(input_size,data_size), 一次读入块大小为30，开启随机打乱
dataset=RandomDataset(input_size,data_size)
rand_loader = DataLoader(dataset=dataset,batch_size=batch_size,shuffle=True) # shuffle开启随机打乱

### 简单模型
为了做一个小demo，我们的模型只是获得了一个输入，执行一个线性操作，然后给一个输出。尽管如果，你可使用DataParallel 在任何模型（CNN，RNN，Capsule Net等）。  

我们放置了一个输出生命在模型中来检测输出和输入张量大小。请注意在batch rank0中的输出。

In [4]:
# simple model
class Net(nn.Module):
    def __init__(self,input_size,output_size):
        super(Net,self).__init__()
        # simple linear layer
        self.fc = nn.Linear(input_size,output_size)
    
    def forward(self,input):
        output = self.fc(input)
        print('Model: input size - {input_size}, output size - {output_size}'.format(input_size=input_size,output_size=output_size))
        return output

### 使用并行处理
**重点**：首先我们需要一个模型的实例，然后验证我们是否拥有多个GPU，如果我们有多个GPU，我们可用nn.DataParallel来包裹我们的模型。然后我们使用model.to(device)把模型放到多个GPU上。

In [5]:
model = Net(input_size,output_size)
# 验证是否有多个GPU
if torch.cuda.device_count() > 1:
    print('We have {num} GPUs.'.formatm(num=torch.cuda.device_count()))
    # 假设有3个GPU，使用并行处理数据的话，就会将维度为[30,x,m,n]的数据分成三个[10,x,m,n],[10,x,m,n],[10,x,m,n]并分别在3个GPU上进行处理数据。
    model = nn.DataParallel(model) # 数据划分并移交给GPUs处理
model.to(device) # 将模型移交给GPUs处理

Net(
  (fc): Linear(in_features=5, out_features=2, bias=True)
)

### 运行模型
查看结果

In [6]:
for _, data in enumerate(tqdm(rand_loader)):
    input = data.to(device)
    output = model(input)

100%|██████████| 4/4 [00:00<00:00, 896.31it/s]

Model: input size - 5, output size - 2
Model: input size - 5, output size - 2
Model: input size - 5, output size - 2
Model: input size - 5, output size - 2


### 结果示例
由于我电脑没有GPU，因此我只能给出一些示例图片了。  
#### 0/1GPU
你只有一个GPU或没有时，当我们获取30个输入和30个输出，模型将期望获得30个输入和30个输出。

![figure.1](https://gitee.com/zyp521/upload_image/raw/master/b3kET1.png)

#### 多个CPUs
如果你有多个GPUs时，你将获得如下的结果：  
两个GPUs：

![figure.2](https://gitee.com/zyp521/upload_image/raw/master/dZiwXd.png)

三个GPUs：

![figure.3](https://gitee.com/zyp521/upload_image/raw/master/4E4lje.png)

## 总结
数据并行自动拆分了你的数据并且将任务发送到多个GPUs上，当每一个模型完成自己的任务之后，DataParallel收集并且合并这些数据，然后返还给你。（能够提高训练速度）